#### Imports 

In [1]:
import pandas as pd
from selenium import webdriver 
import time
from random import shuffle

#### План

- Таблица предпочтений 
- Словарь станция-айди в яндекс.метро
- Брут форс перебором считаем в словарь расстояние в минутах до каждой из станций в таблице
- Наиболее близкая станция - берем ее предпочтение 
- Если одинаковые по времени - пишу "не определились"

In [90]:
# таблица предпочтенй
prf = pd.read_csv('data/preferences.csv', header=None, names=['Station', 'Likes'])

In [91]:
prf.loc[:,'Line'] = [2, 2, 2, 5, 7, 10, 9, 1, 6, 2, 5]

In [92]:
prf

,Station,Likes,Line
0,Сокол,Кофе,2
1,Маяковская,Чай,2
2,Речной вокзал,Кофе,2
3,Белорусская,Чай,5
4,Сходненская,Кофе,7
5,Люблино,Кофе,10
6,Пражская,Чай,9
7,Юго-Западная,Чай,1
8,Калужская,Кофе,6
9,Ховрино,Чай,2


#### Таблица Станция - ID в Яндекс.Метро

In [6]:
from metro_data import LINES, STATIONS

In [36]:
id_table = pd.DataFrame(columns=['id','line','station']) 

In [37]:
for i in STATIONS.keys():
    id_table.loc[i]=[i,STATIONS[i]['line'],STATIONS[i]['name']]

In [47]:
id_table.head()

,id,line,station
1,1,1,Бульвар Рокоссовского
2,2,1,Черкизовская
3,3,1,Преображенская площадь
4,4,1,Сокольники
5,5,1,Красносельская


In [39]:
id_table.to_csv('data/id_table.csv', index=None)

In [48]:
lns = pd.DataFrame(columns=['line','line_name']) 
for i in LINES.keys():
    lns.loc[i]=[i,LINES[i]]
lns.head()

,line,line_name
1,1,Сокольническая линия
2,2,Замоскворецкая линия
3,3,Арбатско-Покровская линия
4,4,Филевская линия
5,5,Кольцевая линия


In [49]:
metro = id_table.merge(lns, on='line', how='left')

In [51]:
metro.head()

,id,line,station,line_name
0,1,1,Бульвар Рокоссовского,Сокольническая линия
1,2,1,Черкизовская,Сокольническая линия
2,3,1,Преображенская площадь,Сокольническая линия
3,4,1,Сокольники,Сокольническая линия
4,5,1,Красносельская,Сокольническая линия


In [50]:
metro.to_csv('data/metro.csv', index=None)

#### Запрос на Яндекс.Метро

In [54]:
# инициализация браузера, в котором будут вычисления
browser = webdriver.Chrome('/Users/devdariya/Downloads/chromedriver')

In [55]:
def route_info(start, stop):
    
    browser.get(f'https://yandex.ru/metro/moscow?from={start}&to={stop}&route=0/') #загружаем страницу
    time.sleep(1)
    
    # выполняем скрипт страницы (т.к. без его выполнения исходный код страницы
    # не содержит необходимую информацию)
    
    browser.execute_script('//yastatic.net/s3/front-maps-static/front-metro/2.11.9/build/index/_index.ru.js') 
    time.sleep(1)
    
    #информация о времени содержится в классе "route-list-item__time" в формате "≈ 10 мин."
    str_time = browser.find_element_by_class_name('masstransit-route-snippet-view__route-duration').text 
    
    #так как маршрут может занять больше часа, обрабатываем и такое время
    #ищем подстроку с числом и преобразуем его в формат int
    if ('м' in str_time)&('ч' in str_time):
        return (int(str_time[str_time.find('≈')+1:str_time.find('ч')-1])*60 
                + int(str_time[str_time.find('ч')+2:str_time.find('м')]))
    elif 'ч' in str_time: 
        return (int(str_time[str_time.find('≈')+1:str_time.find('ч')-1])*60)
    
    return int(str_time[str_time.find('≈')+1:str_time.find('м')])

In [109]:
# brute force
def find_likes(id_new):
    
    routes = pd.DataFrame({'station':prf.Station,
                           'prfr':  prf.Likes,
                            'time':0})
    for i in range(len(prf)):
        id_st = metro[(metro.station==prf.Station[i])&(metro.line==prf.Line[i])].id.values[0]
        routes.loc[routes.station==prf.Station[i], 'time'] = route_info(id_new, id_st)
    
    res = routes[routes.time==min(routes.time)]
    if len(res) > 1:
        if res.prfr.nunique() > 1:
            return -1
        else:
            return res.prfr.unique()
    else:
        return res.prfr.unique()

In [110]:
print('Жители станции метро {} вероятно предпочитают {}'.format(metro[metro.id==49].station.values[0], find_likes(49)[0]))

Жители станции метро Арбатская вероятно предпочитают Чай


#### Интерфейс выбора станции

![Схема в 2018 году](http://news.metro.ru/18/mm20180830.jpg)



In [53]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
 